In [ ]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.safari.options import Options
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.pyplot as plt
from comut import comut
import seaborn as sns
import numpy as np

: 

In [ ]:
plt.rcParams['figure.dpi'] = 150

In [ ]:
safari_options = Options()
safari_options.add_argument("headless")

In [ ]:
baseurl = r'https://www.encodeproject.org/matrix/?type=Experiment&control_type%21=%2A&status=released&perturbed=false&assay_title=TF+ChIP-seq&target.investigated_as=transcription+factor&replicates.library.biosample.donor.organism.scientific_name=Homo+sapiens&biosample_ontology.classification=cell+line&biosample_ontology.term_name='

: 

In [ ]:
# select cell : HepG2, K562, HEK293, GM12878, MCF-7, WTC11, A549, SK-N-SH, HeLa-S3, H1

In [ ]:
cell_lines = []
tfs = []
counts = []

for cellline in ['HepG2', 'K562', 'HEK293', 'GM12878', 'MCF-7', 'WTC11', 'A549', 'SK-N-SH', 'HeLa-S3', 'H1']:
    url = baseurl + cellline
    dr = webdriver.Safari(options=safari_options)
    dr.implicitly_wait(10)
    dr.get(url)
    dr.implicitly_wait(10)
    tfs_div = dr.find_element(By.XPATH,'//*[@id="facet-group-Assay-Experiment"]/div[4]/div/div[2]/ul/div[1]')
    tf_div = tfs_div.find_elements(By.CLASS_NAME, 'facet-term')
    for index, tf in enumerate(tf_div):
        cell_lines.append(cellline)
        tfs.append(tf.find_element(By.CLASS_NAME, 'facet-term__text').text)
        counts.append(tf.find_element(By.CLASS_NAME, 'facet-term__count').text)
    dr.quit()


In [ ]:
tfs_data_encode = pd.DataFrame({
    "Cell" : cell_lines,
    "TF" : tfs,
    "Count" : counts
})

In [ ]:
tfs_data_encode.groupby('Cell').size()

In [ ]:
tfs_family = pd.read_csv('data/tfs_family.csv')
tfs_family_dict = dict(tfs_family[['Name','Family']].values)

# add Unkonw
for i in tfs_data_encode['TF']:
    if i in tfs_family_dict.keys():
        pass
    else:
        tfs_family_dict[i] = "Unknown"



In [ ]:
tfs_data_encode['Family'] = [ tfs_family_dict[i] for i in tfs_data_encode['TF'] ]

In [ ]:
sns.barplot(tfs_data_encode.groupby(['Cell', 'Family']).size().reset_index(name="Count"), x = "Family", y="Count", hue="Cell")
plt.tick_params(axis='x', labelrotation=45)
plt.show()

In [ ]:
tfs_data_encode.groupby(['Cell', 'Family']).size()

In [ ]:
len(tfs_data_encode.drop_duplicates('Family')['Family'])

In [ ]:
comut_data = tfs_data_encode.groupby(['Cell', 'Family']).size().reset_index(name="Count")
comut_data.columns = ['category', 'sample', 'value']
#comut_data['value'] = [ 1/ (1 + np.exp(-i)) for i in comut_data['value'].tolist() ]
comut_data['value'] = "1"

In [ ]:
family_comut = comut.CoMut()
mut_mapping={"1":"#4876FF"}
family_comut.add_categorical_data(comut_data, name='type',mapping=mut_mapping)
#family_comut.add_continuous_data(comut_data, name='type')
family_comut.plot_comut(figsize = (10,3),x_padding = 0.04, y_padding = 0.04)
plt.savefig("results/encode.tf.family.pdf")

In [ ]:
[ 1/ (1 + np.exp(-i)) for i in comut_data['value'].tolist() ]

In [ ]:
np.log10(comut_data['value'].tolist() ) / np.log10(comut_data['value'].max())
comut_data['value'].tolist() 